<a href="https://colab.research.google.com/github/Lakshika-Swarnamali/Diagrams/blob/master/FYP_LSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use ('fivethirtyeight')
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections

import tweepy as tw
import nltk
from nltk.corpus import stopwords
import re
import networkx

import warnings
warnings.filterwarnings("ignore")

sns.set(font_scale=1.5)
sns.set_style("whitegrid")
# packages to store and manipulate data
import pandas as pd
import numpy as np

# plotting packages
import matplotlib.pyplot as plt
import seaborn as sns

# model building package
import sklearn

# package to clean text
import re

In [ ]:
log = pd.read_csv("https://raw.githubusercontent.com/GihanKLG/TwitterCovid19Dataset/master/2020-10-12.csv",encoding='latin1')
log[(log['lang'] == 'en')].to_csv("out.csv", index=False)
log = pd.read_csv('out.csv', lineterminator='\n',encoding='latin1')
log.head()

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,place,possibly_sensitive,retweet_count,retweet_id,retweet_screen_name,source,text,tweet_url,user_created_at,user_screen_name,user_default_profile_image,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name.1,user_statuses_count,user_time_zone,user_urls,user_verified
0,NaN,Sun Oct 11 23:59:54 +0000 2020,NaN,NaN,NaN,0,1315442032651927552,BalutSmithy,1.315403e+18,1.296441e+18,en,NaN,NaN,0,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",@BalutSmithy @StephenJohn59 @jaketapper @kaitl...,https://twitter.com/NorthLapp/status/131544203...,Sun Mar 25 19:25:10 +0000 2012,NorthLapp,False,"I need more followers, I will expose the truth...",1979,1291,1805,28,NaN,Renee J North Lapp,NorthLapp,12503,NaN,NaN,False
1,NaN,Sun Oct 11 23:59:56 +0000 2020,NaN,NaN,NaN,0,1315442037286678529,NaN,NaN,NaN,en,NaN,NaN,1,1.315348e+18,PetraCEsser1,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @PetraCEsser1: @HSRetoucher Thank you. I am...,https://twitter.com/johncoppa44/status/1315442...,Fri Feb 26 09:25:23 +0000 2016,johncoppa44,True,NaN,3031,31,287,1,NaN,john coppa,johncoppa44,7490,NaN,NaN,False
2,NaN,Sun Oct 11 23:59:56 +0000 2020,NaN,NaN,https://twitter.com/onlinelonghorn/status/1315...,0,1315442039870324736,NaN,NaN,NaN,en,NaN,False,0,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",*corona liked this post* https://t.co/3mAPNWNAfK,https://twitter.com/ivythekingrat/status/13154...,Mon Dec 11 16:09:37 +0000 2017,ivythekingrat,False,nope,4001,95,153,0,puerto rico,rat zone,ivythekingrat,1633,NaN,NaN,False
3,NaN,Sun Oct 11 23:59:55 +0000 2020,SidNaaz,NaN,NaN,0,1315442036028260353,NaN,NaN,NaN,en,NaN,NaN,120,1.315334e+18,SidNaaz13Dstny,"<a href=""http://twitter.com/download/android"" ...",RT @SidNaaz13Dstny: Points\n\n1. Salman Sir #S...,https://twitter.com/Tejakshi1/status/131544203...,Sat Jan 04 06:50:54 +0000 2020,Tejakshi1,False,Ã°ÂÂ¤Â¡,57815,282,579,0,NaN,tejakshi shetty,Tejakshi1,39117,NaN,NaN,False
4,NaN,Sun Oct 11 23:59:55 +0000 2020,NaN,NaN,NaN,0,1315442033331507203,NaN,NaN,NaN,en,NaN,NaN,10711,1.315345e+18,CLewandowski_,"<a href=""http://twitter.com/#!/download/ipad"" ...",RT @CLewandowski_: WHO now advising against lo...,https://twitter.com/LoisR52511816/status/13154...,Sun Aug 02 16:10:47 +0000 2020,LoisR52511816,True,NaN,28886,88,277,0,NaN,LoisR,LoisR52511816,13602,NaN,NaN,False


In [ ]:
# importing pandas package
import pandas as pd
 
# making data frame from csv file
data = pd.read_csv("https://raw.githubusercontent.com/GihanKLG/TwitterCovid19Dataset/master/2020-10-12.csv", index_col ="id")
 
# retrieving columns by indexing operator
# first = data["user_name"]
my_column_names = [ 'user_name','text']
df = pd.DataFrame(data=data, columns=my_column_names)
lda = pd.DataFrame(data=data, columns=my_column_names)
lsa = pd.DataFrame(data=data, columns=my_column_names)

# print(first)
# print(first['id', 'user_name','retweet_count','retweet_id'])
print(df)
# df.head(5)

                              user_name                                               text
id                                                                                        
1315442032651927552  Renee J North Lapp  @BalutSmithy @StephenJohn59 @jaketapper @kaitl...
1315442033507684352             🎃 leh 🎃  RT @dougraz: todo dia pensando será se eu ja p...
1315442037286678529          john coppa  RT @PetraCEsser1: @HSRetoucher Thank you. I am...
1315442040172425216      Reivindicación               @PedroAzzola https://t.co/RxL0DAaQaI
1315442039870324736            rat zone   *corona liked this post* https://t.co/3mAPNWNAfK
...                                 ...                                                ...
1315457129835302914     AnnieMoonIsBlue  RT @BillKristol: On President Trump and Dr. Fa...
1315457130460327938        Dardo Boggia  RT @osvaldo_quiroga: Lo entrevisté muchísimas ...
1315457129847889921                Shay    ISOLATION\n\nNot COVID. https://t.co/d4j0jm7j0e

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# pd.set_option("display.max_colwidth", 200)

In [ ]:
# from sklearn.datasets import fetch_20newsgroups

# dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
# documents = dataset.data
# len(documents)

In [ ]:
# dataset.target_names

In [ ]:
# # news_df = pd.DataFrame({'document':documents})
# news_df = pd.DataFrame(df)

# # removing everything except alphabets`
# news_df['clean_doc'] = news_df['text'].str.replace("[^a-zA-Z#]", " ")

# # removing short words
# news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

# # make all text lowercase
# news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [ ]:
# import nltk
# nltk.download('stopwords')

In [ ]:
# from nltk.corpus import stopwords
# stop_words = stopwords.words('english')

# # tokenization
# tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())

# # remove stop-words
# tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

# # de-tokenization
# detokenized_doc = []
# for i in range(len(news_df)):
#     self.adj = {}
#     t = ' '.join(tokenized_doc[i])
#     detokenized_doc.append(t)

# news_df['clean_doc'] = detokenized_doc

In [ ]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

In [ ]:
def remove_links(text):
    '''Takes a string and removes web links from it'''
    text = re.sub(r'http\S+', '', text) # remove http links
    text = re.sub(r'bit.ly/\S+', '', text) # rempve bitly links
    text = text.strip('[link]') # remove [links]
    return text

def remove_users(text):
    '''Takes a string and removes retweet and @user information'''
    text = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', text) # remove retweet
    text = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', text) # remove tweeted at
    return text

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
my_stopwords = nltk.corpus.stopwords.words('english')
word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'

# cleaning master function
def clean_tweet(text, bigrams=False):
    text = remove_users(text)
    text = remove_links(text)
    text = text.lower() # lower case
    text = re.sub('['+my_punctuation + ']+', ' ', text) # strip punctuation
    text = re.sub('\s+', ' ', text) #remove double spacing
    text = re.sub('([0-9]+)', '', text) # remove numbers
    tweet_token_list = [word for word in text.split(' ')
                            if word not in my_stopwords] # remove stopwords

    tweet_token_list = [word_rooter(word) if '#' not in word else word
                        for word in tweet_token_list] # apply word rooter
    if bigrams:
        tweet_token_list = tweet_token_list+[tweet_token_list[i]+'_'+tweet_token_list[i+1]
                                            for i in range(len(tweet_token_list)-1)]
    tweet = ' '.join(tweet_token_list)
    return tweet

In [ ]:
lda['clean_tweet'] = lda.text.apply(clean_tweet)
lda.head(10)

,user_name,text,clean_tweet
id,,,
1315442032651927552,Renee J North Lapp,"@BalutSmithy @StephenJohn59 @jaketapper @kaitlancollins Seems to me that Fauci is also to blame for the mishandling of this pandemic, Birx too! Pres listened to their advice of what to do, when t...",seem fauci also blame mishandl pandem birx pre listen advic pre doctor fauci led team blame fauci trust fauci chang opinion oft
1315442033507684352,🎃 leh 🎃,RT @dougraz: todo dia pensando será se eu ja peguei covid ou não,todo dia pensando será se eu ja peguei covid ou não
1315442037286678529,john coppa,RT @PetraCEsser1: @HSRetoucher Thank you. I am speaking here with tongues of angels.\nThey bring a chip through BLOOD-BRAIN-BARRIER into the…,thank speak tongu angel bring chip blood brain barrier the…
1315442040172425216,Reivindicación,@PedroAzzola https://t.co/RxL0DAaQaI,
1315442039870324736,rat zone,*corona liked this post* https://t.co/3mAPNWNAfK,corona like post
1315442032350027780,ᴮᴱ#LAKERS 2020 NBA Champions #Titans (5-0),https://t.co/VNNyjhzXxt https://t.co/DRIUE2K2VV,
1315442036028260353,tejakshi shetty,RT @SidNaaz13Dstny: Points\n\n1. Salman Sir #SidNaaz shipper h❤️\n\n2. Salman Sir Shehnaaz Biased Shipper h❤️\n\n3. Sidharth ko indirect way mai…,point salman sir #sidnaaz shipper h❤️ salman sir shehnaaz bias shipper h❤️ sidharth ko indirect way mai…
1315442038142390274,Angel Duarte Hurtado,RT @AristeguiOnline: Paga México más de 180 millones de dls. para garantizar vacuna contra Covid-19: SRE https://t.co/VaRI57CziU https://t.…,paga méxico má de millon de dl para garantizar vacuna contra covid sre
1315442033331507203,LoisR,RT @CLewandowski_: WHO now advising against lockdowns because they disproportionately hurt the poor—which is what ⁦@realDonaldTrump⁩ argued…,advis lockdown disproportion hurt poor—which ⁦⁩ argued…


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # keep top 1000 terms 
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(lda['clean_tweet'])

X.shape # check shape of the document-term matrix

(166536, 1000)

In [ ]:
from sklearn.decomposition import TruncatedSVD

# SVD represent documents and terms in vectors 
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)

svd_model.fit(X)

len(svd_model.components_)

20

In [ ]:
terms = vectorizer.get_feature_names()

for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
    print("Topic "+str(i)+": ")
    for t in sorted_terms:
        print(t[0])
        print(" ")

Topic 0: 
like
 
know
 
people
 
think
 
good
 
time
 
thanks
 
Topic 1: 
thanks
 
windows
 
card
 
drive
 
mail
 
file
 
advance
 
Topic 2: 
game
 
team
 
year
 
games
 
season
 
players
 
good
 
Topic 3: 
drive
 
scsi
 
disk
 
hard
 
card
 
drives
 
problem
 
Topic 4: 
windows
 
file
 
window
 
files
 
program
 
using
 
problem
 
Topic 5: 
government
 
chip
 
mail
 
space
 
information
 
encryption
 
data
 
Topic 6: 
like
 
bike
 
know
 
chip
 
sounds
 
looks
 
look
 
Topic 7: 
card
 
sale
 
video
 
offer
 
monitor
 
price
 
jesus
 
Topic 8: 
know
 
card
 
chip
 
video
 
government
 
people
 
clipper
 
Topic 9: 
good
 
know
 
time
 
bike
 
jesus
 
problem
 
work
 
Topic 10: 
think
 
chip
 
good
 
thanks
 
clipper
 
need
 
encryption
 
Topic 11: 
thanks
 
right
 
problem
 
good
 
bike
 
time
 
window
 
Topic 12: 
good
 
people
 
windows
 
know
 
file
 
sale
 
files
 
Topic 13: 
space
 
think
 
know
 
nasa
 
problem
 
year
 
israel
 
Topic 14: 
space
 
good
 
card
 
people
 
time
 
nas

In [ ]:
pip install 'umap-learn==0.3.10'

     |████████████████████████████████| 40kB 4.6MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.3.10-cp37-none-any.whl size=38882 sha256=375f1805a7d8cc608f81ef9435c43627ef6f534095b8babbbd6d14e4443a6945
  Stored in directory: /root/.cache/pip/wheels/d0/f8/d5/8e3af3ee957feb9b403a060ebe72f7561887fef9dea658326e
Successfully built umap-learn


In [ ]:
import umap

X_topics = svd_model.fit_transform(X)
embedding = umap.UMAP(n_neighbors=150, min_dist=0.5, random_state=12).fit_transform(X_topics)

plt.figure(figsize=(7,5))
plt.scatter(embedding[:, 0], embedding[:, 1], 
c = dataset.target,
s = 10, # size
edgecolor='none'
)
plt.show()

In [ ]:
def remove_links(text):
    '''Takes a string and removes web links from it'''
    text = re.sub(r'http\S+', '', text) # remove http links
    text = re.sub(r'bit.ly/\S+', '', text) # rempve bitly links
    text = text.strip('[link]') # remove [links]
    return text

def remove_users(text):
    '''Takes a string and removes retweet and @user information'''
    text = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', text) # remove retweet
    text = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', text) # remove tweeted at
    return text

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
my_stopwords = nltk.corpus.stopwords.words('english')
word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'

# cleaning master function
def clean_tweet(text, bigrams=False):
    text = remove_users(text)
    text = remove_links(text)
    text = text.lower() # lower case
    text = re.sub('['+my_punctuation + ']+', ' ', text) # strip punctuation
    text = re.sub('\s+', ' ', text) #remove double spacing
    text = re.sub('([0-9]+)', '', text) # remove numbers
    tweet_token_list = [word for word in text.split(' ')
                            if word not in my_stopwords] # remove stopwords

    tweet_token_list = [word_rooter(word) if '#' not in word else word
                        for word in tweet_token_list] # apply word rooter
    if bigrams:
        tweet_token_list = tweet_token_list+[tweet_token_list[i]+'_'+tweet_token_list[i+1]
                                            for i in range(len(tweet_token_list)-1)]
    tweet = ' '.join(tweet_token_list)
    return tweet

In [ ]:
lda['clean_tweet'] = lda.text.apply(clean_tweet)
lda.head(10)

,user_name,text,clean_tweet
id,,,
1315442032651927552,Renee J North Lapp,@BalutSmithy @StephenJohn59 @jaketapper @kaitl...,seem fauci also blame mishandl pandem birx pr...
1315442033507684352,🎃 leh 🎃,RT @dougraz: todo dia pensando será se eu ja p...,todo dia pensando será se eu ja peguei covid ...
1315442037286678529,john coppa,RT @PetraCEsser1: @HSRetoucher Thank you. I am...,thank speak tongu angel bring chip blood brai...
1315442040172425216,Reivindicación,@PedroAzzola https://t.co/RxL0DAaQaI,
1315442039870324736,rat zone,*corona liked this post* https://t.co/3mAPNWNAfK,corona like post
1315442032350027780,ᴮᴱ#LAKERS 2020 NBA Champions #Titans (5-0),https://t.co/VNNyjhzXxt https://t.co/DRIUE2K2VV,
1315442036028260353,tejakshi shetty,RT @SidNaaz13Dstny: Points\n\n1. Salman Sir #S...,point salman sir #sidnaaz shipper h❤️ salma...
1315442038142390274,Angel Duarte Hurtado,RT @AristeguiOnline: Paga México más de 180 mi...,paga méxico má de millon de dl para garantiz...
1315442033331507203,LoisR,RT @CLewandowski_: WHO now advising against lo...,advis lockdown disproportion hurt poor—which ...


In [ ]:
# Applying Topic Modelling
from sklearn.feature_extraction.text import CountVectorizer

# the vectorizer object will be used to transform text to vector form
vectorizer = CountVectorizer(max_df=0.9, min_df=25, token_pattern='\w+|\$[\d\.]+|\S+')

# apply transformation
tf = vectorizer.fit_transform(lda['clean_tweet']).toarray()

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names()
# tf_feature_names

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 10

model = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)

In [ ]:
model.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [ ]:
def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)

In [ ]:
no_top_words = 10
display_topics(model, tf_feature_names, no_top_words)

,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights,Topic 5 words,Topic 5 weights,Topic 6 words,Topic 6 weights,Topic 7 words,Topic 7 weights,Topic 8 words,Topic 8 weights,Topic 9 words,Topic 9 weights
0,…,8684.6,pandem,7489.9,…,5993.9,…,4205.1,…,13692.0,…,4977.5,de,19647.1,covid,10327.5,…,13180.9,…,12499.0
1,de,5287.7,go,5215.8,lockdown,2808.7,trump,3540.0,covid,13658.5,de,2061.3,la,12528.0,…,8515.6,covid,9242.6,mask,8518.4
2,comment,3803.1,covid,3730.4,end,1752.9,fauci,3324.2,’,5297.1,lockdown,1882.5,el,10200.0,new,3406.1,trump,7783.7,wear,8320.0
3,amp,3731.8,y,3490.1,la,1617.4,respons,2727.1,’t,4764.4,covid,1851.8,covid,9239.6,trump,3403.1,’,7099.2,live,6440.9
4,en,3725.1,hear,3462.0,call,1452.4,say,2693.7,peopl,3890.5,e,1712.1,…,8948.0,american,3316.3,twitter,3011.7,get,5534.1
5,campaign,3665.6,’all,3364.1,cdc,1429.4,dr,2419.9,pandem,3378.3,que,1696.8,que,8931.3,death,3238.1,test,2868.3,let,5444.3
6,latest,3548.1,song,3288.1,sign,1426.3,ad,2344.8,like,3353.6,current,1599.5,en,6718.1,die,3231.9,he,2856.1,person,5308.0
7,fauci,3491.5,…,2490.4,coronaviru,1357.5,coronaviru,2192.2,get,3315.6,com,1553.7,por,4229.1,case,2977.2,dr,2705.2,pandem,5113.0
8,use,3445.2,flu,2030.1,scientist,1257.8,new,1731.6,trump,3244.4,uma,1513.1,lo,4081.5,day,2647.0,immun,2514.2,save,4929.1
9,dr,3443.5,tell,2020.5,chri,1152.1,campaign,1724.5,amp,2961.3,pode,1483.1,se,3793.2,one,2384.9,coronaviru,2472.9,choic,4434.1


In [ ]:
# df['is_retweet'] = df['text'].apply(lambda x: x[:2]=='RT')
# df['is_retweet'].sum()  # number of retweets

In [ ]:
# df.groupby(['text']).size().reset_index(name='counts')\
#   .sort_values('counts', ascending=False).head(10)

In [ ]:
# tweets = pd.DataFrame(df)
# def find_mentioned(tweet):
#     '''This function will extract the twitter handles of people mentioned in the tweet'''
#     return re.findall('(?<!RT\s)(@[A-Za-z]+[A-Za-z0-9-_]+)', tweet)
# def find_hashtags(tweet):
#     '''This function will extract hashtags'''
#     return re.findall('(#[A-Za-z]+[A-Za-z0-9-_]+)', tweet)
# # make new columns for mentioned usernames and hashtags
# tweets['mentioned'] = tweets.text.apply(find_mentioned)
# tweets['hashtags'] = tweets.text.apply(find_hashtags)
# tweets.head(5)

In [ ]:
# # number of unique hashtags
# flattened_hashtags_df = pd.DataFrame(tweets)
# flattened_hashtags_df['hashtags'].unique().size
# flattened_hashtags_df.head(5)

In [ ]:
# popular_hashtags = list(log['hashtags'])
# popular_hashtags = [x for x in all_tweets if str(x) != 'nan']

# popular_hashtags[:5]

In [ ]:
# popular_hash['hashtag'].unique().size
# popular_hashtags[:5]

In [ ]:
# # take hashtags which appear at least this amount of times
# min_appearance = 1000
# # find popular hashtags - make into python set for efficiency
# popular_hashtags_set = set(popular_hashtags[popular_hashtags.counts>=min_appearance]['hashtags'])
# popular_hashtags_set[:5]

In [ ]:
# my_stopwords = nltk.corpus.stopwords.words('english')
# word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
# my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'
# # cleaning master function
# def clean_tweet(tweet, bigrams=False):
#     tweet = remove_users(tweet)
#     tweet = remove_links(tweet)
#     tweet = tweet.lower() # lower case
#     tweet = re.sub('['+my_punctuation + ']+', ' ', tweet) # strip punctuation
#     tweet = re.sub('\s+', ' ', tweet) #remove double spacing
#     tweet = re.sub('([0-9]+)', '', tweet) # remove numbers
#     tweet_token_list = [word for word in tweet.split(' ')
#                             if word not in my_stopwords] # remove stopwords
# tweet_token_list = [word_rooter(word) if '#' not in word else word
#                         for word in tweet_token_list] # apply word rooter
# if bigrams:
#         tweet_token_list = tweet_token_list+[tweet_token_list[i]+'_'+tweet_token_list[i+1]
#                                             for i in range(len(tweet_token_list)-1)]
#     tweet = ' '.join(tweet_token_list)
#     return tweet
# tweets['clean_tweet'] = tweets.full_text.apply(clean_tweet)

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer
# # the vectorizer object will be used to transform text to vector form
# vectorizer = CountVectorizer(max_df=0.9, min_df=100, token_pattern='\w+|\$[\d\.]+|\S+')
# # apply transformation
# tf = vectorizer.fit_transform(tweets['text']) #.toarray()
# # tf_feature_names tells us what word each column in the matric represents
# tf_feature_names = vectorizer.get_feature_names()
# tf.shape # --> (200000, 2296)

(166536, 3152)

In [ ]:
# from sklearn.decomposition import LatentDirichletAllocation
# number_of_topics = 10
# model = LatentDirichletAllocation(n_components=number_of_topics, random_state=45) # random state for reproducibility
# # Fit data to model
# model.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=45, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)